In [ ]:
!pip install gluonnlp
# !pip install mxnet
!pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'
# !pip install git+https://git@github.com/SKTBrain/KoBERT.git@master
!pip install transformers
b!pip install sentencepiece
!pip install kobert-transformers
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 344 kB 8.8 MB/s 


In [6]:
!nvidia-smi

Tue Jun 28 14:17:15 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.33.01    Driver Version: 440.33.01    CUDA Version: 10.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla V100-PCIE...  On   | 00000000:00:06.0 Off |                    0 |
| N/A   27C    P0    41W / 250W |  14142MiB / 32510MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   1  Tesla V100-PCIE...  On   | 00000000:00:07.0 Off |                    0 |
| N/A   25C    P0    41W / 250W |   7195MiB / 32510MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
        

In [1]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm.notebook import tqdm
import pandas as pd
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup
from kobert_tokenizer import KoBERTTokenizer
from sklearn.model_selection import train_test_split

2022-06-28 14:19:51.232215: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1


In [2]:
#csv 형식의 training 데이터를 로드합니다.
import pandas as pd 

df = pd.read_csv('data/train.csv')
test_df = pd.read_csv('data/test.csv')

#데이터 살펴보기 위해 데이터 최상단의 5줄을 표시합니다.
df.head() 

,id,reviews,target
0,0,조아요 처음구입 싸게햇어요,2
1,1,생각보다 잘 안돼요 매지 바른지 하루밖에 안됐는데ㅠㅠ 25천원가량 주고 사기 너무 ...,1
2,2,디자인은괜찮은데 상품이 금이가서 교환했는데 두번째받은상품도 까져있고 안쪽에 금이가져...,2
3,3,기전에 이 제품말고 이마트 트레이더스에서만 팔던 프리미엄 제품을 사용했었습니다. 샘...,2
4,4,튼튼하고 손목을 잘 받쳐주네요~,5


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25000 entries, 0 to 24999
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   id       25000 non-null  int64 
 1   reviews  25000 non-null  object
 2   target   25000 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 586.1+ KB


In [4]:
tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')

In [5]:
def text_to_token(df:pd.DataFrame):
  input_ids = list()
  token_type_ids = list()
  attention_mask = list()

  for i in range(len(df)):
    temp = tokenizer.encode_plus(df['reviews'].loc[i], add_special_tokens=True,
            pad_to_max_length=True,
            return_attention_mask = True, 
            return_token_type_ids=True,
            truncation=True)
    input_ids.append(temp['input_ids'])
    token_type_ids.append(temp['token_type_ids'])
    attention_mask.append(temp['attention_mask'])

  return input_ids, token_type_ids, attention_mask

df['input_ids'], df['token_type_ids'], df['attention_mask'] = text_to_token(df)

/home/ubuntu/anaconda3/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:2126: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [6]:
df[['input_ids', 'token_type_ids', 'attention_mask']].head()

,input_ids,token_type_ids,attention_mask
0,"[2, 4162, 6797, 6999, 4468, 5495, 7138, 3076, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]"
1,"[2, 2705, 6371, 3942, 3135, 5876, 6999, 1986, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
2,"[2, 1831, 7086, 5484, 7396, 7086, 5850, 2683, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
3,"[2, 1258, 7210, 3647, 4158, 6160, 5439, 3689, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
4,"[2, 517, 7668, 7668, 7788, 2860, 6217, 7088, 3...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]"


In [7]:
## CPU
# device = torch.device("cpu")

## GPU
device = torch.device("cuda:0")

In [8]:
bertmodel, vocab = get_pytorch_kobert_model(cachedir=".cache")

using cached model. /home/ubuntu/Dacon/world/Summer/shopping3/.cache/kobert_v1.zip
using cached model. /home/ubuntu/Dacon/world/Summer/shopping3/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


AttributeError: type object 'BERTVocab' has no attribute 'from_sentencepiece'

In [ ]:
my_dict = {}
for i in range(len(df['target'].unique())):
  my_dict[df['target'].unique()[i]] = i

In [ ]:
my_dict

{1: 1, 2: 0, 4: 3, 5: 2}

In [ ]:
def target_to_label(text:str):
  return my_dict[text]

df['target_label'] = df['target'].map(target_to_label)

In [ ]:
df[['target', 'target_label']].head()

,target,target_label
0,2,0
1,1,1
2,2,0
3,2,0
4,5,2


In [ ]:
def remove_enter(text:str):
  return text.replace('\n',"")

df['reviews'] = df['reviews'].map(remove_enter)

In [ ]:
df['reviews'].loc[0]

'조아요 처음구입 싸게햇어요'

In [ ]:
dataset_train, dataset_test = train_test_split(df[['id', 'reviews', 'target_label']], test_size=0.2, random_state=42, shuffle=True)
dataset_train.to_csv('dataset_train.txt', index=False, header=True, sep='\t', encoding='utf8')
dataset_test.to_csv('dataset_test.txt', index=False, header=True, sep='\t', encoding='utf8')

In [ ]:
dataset_train = nlp.data.TSVDataset('/content/dataset_train.txt', field_indices=[1,2], num_discard_samples=1)
dataset_test = nlp.data.TSVDataset('/content/dataset_test.txt', field_indices=[1,2], num_discard_samples=1)

In [ ]:
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [ ]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [ ]:
## Setting parameters
max_len = 140
batch_size = 8
warmup_ratio = 0.1
num_epochs = 20
max_grad_norm = 1
log_interval = 200
learning_rate =  3e-4

In [ ]:
data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)

In [ ]:
next(iter(data_train))

(array([   2,  517, 6994, 6150, 7848, 6559, 1900, 6431, 3135, 6779, 5761,
        6999,  517,   46, 1458,  517, 5591, 6738, 5591, 6738, 7850, 2643,
        7096,  517, 7386, 6060, 6999,  517,   54, 1210, 1165, 5850, 5512,
        5850, 1633, 5660, 5330,  781, 6003, 7245, 6553,  517, 5994, 5411,
        4297, 5703,  517,   54, 2207, 7748, 7728, 7178, 5213, 6855, 1189,
        4955, 7848, 6150, 1633, 7472, 3087, 5591, 6393, 6999,  517, 6896,
        7982,  517,  463,  517,  463,    3,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1], dtype=int32),
 array(72, dtype=int32),
 array([0, 0, 0,

In [ ]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=2)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=2)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [ ]:
len(df['target_label'].unique())

4

In [ ]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=4,
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [ ]:
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

In [ ]:
# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

In [ ]:
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


In [ ]:
t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

In [ ]:
scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

In [ ]:
from sklearn.metrics import top_k_accuracy_score

def calc_accuracy(X,Y):
    #max_vals, max_indices = torch.max(X, 1)
    #train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    max_vals, max_indices = torch.max(X, 1)
    train_acc = top_k_accuracy_score(max_indices, max_vals, k=2, normalize=True, sample_weight=None, labels=None)
    return train_acc

In [ ]:
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

In [ ]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in tqdm(enumerate(train_dataloader), total=len(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in tqdm(enumerate(test_dataloader), total=len(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))
    # torch.cuda.empty_cache()

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


  0%|          | 0/2500 [00:00<?, ?it/s]

epoch 1 batch id 1 loss 1.3324830532073975 train acc 0.25
epoch 1 batch id 201 loss 1.2405976057052612 train acc 0.35945273631840796
epoch 1 batch id 401 loss 1.1525431871414185 train acc 0.442643391521197
epoch 1 batch id 601 loss 1.1301805973052979 train acc 0.49896006655574043
epoch 1 batch id 801 loss 0.944766640663147 train acc 0.5182584269662921
epoch 1 batch id 1001 loss 1.5092244148254395 train acc 0.534965034965035
epoch 1 batch id 1201 loss 1.175006628036499 train acc 0.5428809325562032
epoch 1 batch id 1401 loss 1.5738943815231323 train acc 0.548893647394718
epoch 1 batch id 1601 loss 1.3284095525741577 train acc 0.5539506558401
epoch 1 batch id 1801 loss 0.5344020128250122 train acc 0.5613548028872848
epoch 1 batch id 2001 loss 1.3862075805664062 train acc 0.566904047976012
epoch 1 batch id 2201 loss 0.7190230488777161 train acc 0.5704225352112676
epoch 1 batch id 2401 loss 1.096895456314087 train acc 0.5666909620991254
epoch 1 train acc 0.56785


  0%|          | 0/625 [00:00<?, ?it/s]

epoch 1 test acc 0.614


  0%|          | 0/2500 [00:00<?, ?it/s]

epoch 2 batch id 1 loss 0.9523444175720215 train acc 0.625
epoch 2 batch id 201 loss 1.258042812347412 train acc 0.5827114427860697


In [ ]:
print(test_acc / (batch_id+1))